# Create a TensorFlow Inference Engine
In this section, we will create an inference engine wrapper, a class that will get image data as input, analyze it, and return the analysis result.

## Get Global Variables

First, read the previously stored variables. We need the name of the directory that will be used to store our ML solution files. If this directory does not exist, we will create a directory with a specified directory name.

In [ ]:
import sys
sys.path.append('../common')
from env_variables import *

import os
if not os.path.exists(lvaExtensionPath):
    os.mkdir(lvaExtensionPath)

## Create Inference Engine Wrapper
Next, we will create a class that with different properties and methods to help scoring and analyzing data from an image.

> <span style="color:red; font-weight: bold"> [!IMPORTANT] </span>
> Specific to this sample, we are using the TensorFlow model you exported from Custom Vision. As you can see from the code below,  
> * The YOLOv3 model accepts only raw image bytes with 416 by 416 in size.  
> * Because we expect the `score` method to receive raw bytes of this size (416x416), we have statically coded the image size into our code. If the image is not 416x416 float32, then the code will crash.
> * Why do we statically code the image size? LVA sends video frames to the `score` endpoint. In fact, LVA can send any image size and format. Since LVA can send images with 416x416 size, we do not need to spend additional compute cycles for re-sizing an image.

In [ ]:
%%writefile $lvaExtensionPath/score.py
import threading
from PIL import Image
import numpy as np
import io
import tensorflow as tf
import json
import logging
import os
import linecache
import sys
import math
from object_detection import ObjectDetection

logging.basicConfig(level=logging.DEBUG)

def PrintGetExceptionDetails():
    exType, exValue, exTraceback = sys.exc_info()

    tbFrame = exTraceback.tb_frame
    lineNo = exTraceback.tb_lineno
    fileName = tbFrame.f_code.co_filename

    linecache.checkcache(fileName)
    line = linecache.getline(fileName, lineNo, tbFrame.f_globals)

    exMessage = '[LVAX] Exception:\n\tFile name: {0}\n\tLine number: {1}\n\tLine: {2}\n\tValue: {3}'.format(fileName, lineNo, line.strip(), exValue)

    logging.info(exMessage)

class TFObjectDetection(ObjectDetection):
    """Object Detection class for TensorFlow"""

    def __init__(self, graph_def, labels, prob_threshold, max_detections):
        super(TFObjectDetection, self).__init__(labels, prob_threshold, max_detections)
        self.graph = tf.compat.v1.Graph()
        with self.graph.as_default():
            input_data = tf.compat.v1.placeholder(tf.float32, [1, None, None, 3], name='Placeholder')
            tf.import_graph_def(graph_def, input_map={"Placeholder:0": input_data}, name="")

    def predict(self, preprocessed_image):
        inputs = np.array(preprocessed_image, dtype=np.float)[:, :, (2, 1, 0)]  # RGB -> BGR

        with tf.compat.v1.Session(graph=self.graph) as sess:
            output_tensor = sess.graph.get_tensor_by_name('model_outputs:0')
            outputs = sess.run(output_tensor, {'Placeholder:0': inputs[np.newaxis, ...]})
            return outputs[0]

class MLModel:
    
    def __init__(self):
        try:
            self._modelFileName = 'model.pb'
            self._labelFileName = 'labels.txt'
            self._lock = threading.Lock()
            self.prob_threshold = 0.1
            self.max_detections = 20

            graph_def = tf.compat.v1.GraphDef()
            with tf.io.gfile.GFile(self._modelFileName, 'rb') as f:
                graph_def.ParseFromString(f.read())

            # Load labels
            with open(self._labelFileName, 'r') as f:
                labels = [l.strip() for l in f.readlines()]

            self.od_model = TFObjectDetection(graph_def, labels, self.prob_threshold, self.max_detections)

        except:
            PrintGetExceptionDetails()

 
    def Score(self, pilImage):
        try:
            with self._lock:
                predictions = self.od_model.predict_image(pilImage)

                return predictions

        except:
            PrintGetExceptionDetails()

The `score` method of the inference engine class above will return a dictionary of inferences in the following form:

```
{
  "type": "entity",
  "entity": {
    "tag": {
      "value": "delivery truck",
      "confidence": "0.9596136"
    },
    "box": {
      "l": "0.69242793",
      "t": "0.3647236",
      "w": "0.08401036",
      "h": "0.07765585"
    }
  }
},
{
  "type": "entity",
  "entity": {
    "tag": {
      "value": "delivery truck",
      "confidence": "0.92975134"
    },
    "box": {
      "l": "0.5211431",
      "t": "0.44633362",
      "w": "0.16630614",
      "h": "0.12689856"
    }
  }
}
```

## Next Steps
If all the code cells above have successfully finished running, return to the Readme page to continue.   